In [5]:
from PIL import Image
import csv
import os
from torch.utils.data import Dataset
import pandas as pd
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch
import cv2
import numpy as np
import torchvision.transforms as transforms
from torchvision import models as models
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset
from numpy import argmax
from sklearn.model_selection import train_test_split

#imagepath = 'C:/Users/thoma/Documents/Softwareudvikling/Deep_learning/Eksamensprojekt/kode/DeepLearning/DatasætDeeplearning/license_plates_detection_train/'
#csvpath = 'C:/Users/thoma/Documents/Softwareudvikling/Deep_learning/Eksamensprojekt/kode/DeepLearning/DatasætDeeplearning/license_plates_detection_train.csv'
#croppedImagesPath = 'C:/Users/thoma/Documents/Softwareudvikling/Deep_learning/Eksamensprojekt/kode/DeepLearning/DatasætDeeplearning/croppedImages/'
#trainingdatacsvpath = 'C:/Users/thoma/Documents/Softwareudvikling/Deep_learning/Eksamensprojekt/kode/DeepLearning/trainingdata.csv'
#predictioncsvpath = 'C:/Users/thoma/Documents/Softwareudvikling/Deep_learning/Eksamensprojekt/kode/DeepLearning/DatasætDeeplearning/license_plates_recognition_train.csv'

imagepath = 'C:/Users/chris/source/repos//DeepLearning/DatasætDeeplearning/license_plates_detection_train/'
csvpath = 'C:/Users//chris//source/repos//DeepLearning/DatasætDeeplearning/license_plates_detection_train.csv'
croppedImagesPath = 'C:/Users//chris//source/repos//DeepLearning/DatasætDeeplearning/croppedImages/'
trainingdatacsvpath = 'C:/Users/chris/source/repos//DeepLearning/DatasætDeeplearning/trainingdata.csv'
predictioncsvpath = 'C:/Users//chris/source/repos//DeepLearning/DatasætDeeplearning/license_plates_recognition_train.csv'


In [6]:
#Opgave 1
class ImageData:
    def __init__(self, path, ymin, xmin, ymax, xmax):
        self.path = path
        self.xmin = xmin      
        self.ymin = ymin        
        self.xmax = xmax        
        self.ymax = ymax

In [7]:
def read_image_data():
    file = open(csvpath)
    csvreader = csv.reader(file)
    rows = []
    next(csvreader)
    for row in csvreader:
            id = ImageData(row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]))
            rows.append(id)
    file.close()
    return rows


In [8]:
os.makedirs(croppedImagesPath, exist_ok=True)

def crop_images():
    imageData = read_image_data()

    for id in imageData:
        im = Image.open(imagepath + id.path)
        im = im.crop( (id.xmin, id.ymin, id.xmax, id.ymax) )
        im.save(croppedImagesPath + id.path) 
        im.close()

crop_images()

In [9]:
#Opgave 2
class PredictionData:
    def __init__(self, path, licenseplate):
        self.path = path
        self.licenseplate = licenseplate

In [10]:
def read_prediction_data():
    file = open(predictioncsvpath)
    csvreader = csv.reader(file)
    rows = []
    next(csvreader)
    for row in csvreader:
            pd = PredictionData(row[0], row[1])
            rows.append(pd)
    file.close()
    return rows


In [11]:
def onehot_encoding():
    predictiondata = read_prediction_data()
    numbers = '0123456789'
    onehot_encoded = list()
    header = ['img_id', 'text', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

    with open(trainingdatacsvpath, 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
       

        for data in predictiondata:
            # Removes T and N from licenseplate
            datatrimmed = data.licenseplate.replace('T', '')
            datatrimmed = datatrimmed.replace('N', '')

            char_to_int = dict((c, i) for i, c in enumerate(numbers))
            # integer encode input data
            integer_encoded = [char_to_int[char] for char in datatrimmed]
            #print(integer_encoded)
            # one hot encode
            letter = [0 for _ in range(len(numbers))]
            for value in integer_encoded:
                letter[value] = 1
            
            letter.insert(0, data.path)
            letter.insert(1, data.licenseplate)
            writer.writerow(letter)
            onehot_encoded.append(letter)

onehot_encoding() 
    

In [12]:
#Opgave 3
with open(trainingdatacsvpath) as trainingcsv:
    next(trainingcsv)
    file_read = csv.reader(trainingcsv)
    array = list(file_read)

train, test = train_test_split(array,test_size=0.25,random_state=10) #seed er 10
header = ['img_id', 'text', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

pd.DataFrame(test).to_csv("test.csv",header=header,index=False)
pd.DataFrame(train).to_csv("train.csv",header=header,index=False)


In [17]:
#Opgave 4
np.histogram(test)

TypeError: cannot perform reduce with flexible type